In [1]:
import numpy as np
import pandas as pd
# import modin.pandas as pd

In [2]:
feature_path ='./feature/'
res_path = './res/'
data_path = './data/'
second_round_path = './second round/'

In [3]:
transaction_df = pd.read_csv(data_path+'transaction_train_new.csv')
operation_df =  pd.read_csv(data_path+'operation_train_new.csv')
label = pd.read_csv(data_path+'tag_train_new.csv')

transaction_test = pd.read_csv(data_path+'transaction_round1_new.csv')
operation_test = pd.read_csv(data_path+'operation_round1_new.csv')

# transaction_test = pd.read_csv(second_round_path+'test_transaction_round2.csv')
# operation_test = pd.read_csv(second_round_path+'test_operation_round2.csv')

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# Delete abnormal data
abnormal_device1 = [name for name, count in operation_df.groupby('device1').size().iteritems() if count > 20000]

In [5]:
# operation_df = operation_df[~operation_df.device1.isin(abnormal_device1)]
# transaction_df = transaction_df[~transaction_df.device1.isin(abnormal_device1)]


In [6]:
def merge_count(df1,df2,columns,value,cname):
    add = df1.groupby(columns)[value].count().reset_index().rename(columns = {value:cname})
    df2=df2.merge(add,on=columns,how="left")
    del add
    return df2

def merge_nunique(df1,df2,columns,value,cname):
    add = df1.groupby(columns)[value].nunique().reset_index().rename(columns = {value:cname})
    df2=df2.merge(add,on=columns,how="left")
    del add
    return df2

def merge_value_count(df1,df2,col, name=None, divisor=None):
    if name is None:
        name = col[1]
    tmp = df1.groupby(col).size().reset_index().rename(columns = {0:'cnt'})
    df = tmp.pivot(index=col[0],columns=col[1],values='cnt').reset_index()
    if divisor is not None:
        df = df.apply(lambda x: [x.iloc[0], *(x.iloc[1:] / divisor[x.iloc[0]])], axis=1, result_type='broadcast')
    cname = [col[0]]
    for index in range(1,len(df.columns)):
        cname.append(str(name)+'_'+str(df.columns[index]))
    df.columns=cname
    df = df.fillna(0)
    df2 = df2.merge(df,on=str(col[0]),how='left')
    del df,df1
    return df2

In [7]:
labels_dict = {}
for i, row in label.iterrows():
    labels_dict[row['UID']] = row['Tag']
    
def prepare_ratio_data(transaction_df, label, name):
    col = transaction_df.groupby('UID')[name]
    category_sum = dict()
    category_black_count = dict()
    def manage_categories(m):
        uid = m.name
        m = m.unique()
        for c_id in m:
            category_sum[c_id] = category_sum.get(c_id, 0) + 1
            if labels_dict[uid] == 1:
                category_black_count[c_id] = category_black_count.get(c_id, 0) + 1
    col.apply(manage_categories)
        
    return {m_id: category_black_count.get(m_id, 0) / m_count for m_id, m_count in category_sum.items()}
        


In [8]:

black_ratio = {
    'merchant': prepare_ratio_data(transaction_df, label, 'merchant'),
    'amt_src1': prepare_ratio_data(transaction_df, label, 'amt_src1'),
    'trans_device2': prepare_ratio_data(transaction_df, label, 'device2'),
    'op_device2': prepare_ratio_data(operation_df, label, 'device2'),
    'acc_id2': prepare_ratio_data(transaction_df, label, 'acc_id2'),
    'acc_id3': prepare_ratio_data(transaction_df, label, 'acc_id3'),
}
black_avg = {
    k: np.mean([ratio.get(m, 0) for m in ratio.keys()]) for k, ratio in black_ratio.items()
}

In [9]:

def get_black_ratio(df, df_name, out_name):
    df['%s_black_ratio'%out_name] = df[df_name].apply(lambda x: black_ratio[out_name].get(x, black_avg[out_name]))
    tmp = df.groupby('UID')['%s_black_ratio'%out_name].agg([max, min, np.mean]).reset_index()
    tmp.columns = ['UID', 'max_%s_black_ratio'%out_name, "min_%s_black_ratio"%out_name, "avg_%s_black_ratio"%out_name]
    return tmp


In [209]:
def getPopularityCount(df, df_name, out_name):
    categories = df[df_name].unique()
    counts = {c: 0 for c in categories}
    counts[np.nan] = np.nan
    groups = df[['UID', df_name]].groupby('UID')
    for uid, frame in groups:
        for val in frame[df_name].unique():
            counts[val] += 1
    counts_list = [counts[v] for v in df[df_name]]
    df[out_name] = counts_list
    res = df.groupby('UID')[out_name].agg([max,min,np.mean]).reset_index()
    res.columns = ['UID', '%s_max'%out_name, '%s_min'%out_name, '%s_max'%out_name]
    return res

In [10]:
(transaction_test.groupby('day').size()[:10],
 transaction_df.groupby('day').size()[:10])

(day
 1     2620
 2     3092
 3     3091
 4     2869
 5     3473
 6     8619
 7     6821
 8     3637
 9     2580
 10    2731
 dtype: int64, day
 1     17767
 2     11241
 3      3086
 4      2826
 5      4676
 6      3309
 7      3437
 8     20130
 9     10085
 10     3304
 dtype: int64)

In [11]:
train_weekday = lambda x: (x + 5) % 7
test_weekday = lambda x: x % 7

In [12]:
def add_week_day_features(dataframe, weekday_func):
    days = dataframe['day'].apply(lambda x: weekday_func(x))
    days = days.replace(0, 7)
    dataframe['day_in_week'] = days
    return dataframe
operation_df = add_week_day_features(operation_df, train_weekday)
transaction_df = add_week_day_features(transaction_df, train_weekday)
operation_test = add_week_day_features(operation_test, test_weekday)
transaction_test = add_week_day_features(transaction_test, test_weekday)

In [216]:
import geohash
def gap(x):
    valid = x.dropna()
    if not valid.empty:
        return max(valid) - min(valid)
    else:
        return 0

def get_op_stats_fea(operation_df):
    op_counts = operation_df.groupby('UID').size()
    
    #op_day
    op_fea = operation_df[['UID']].drop_duplicates()
    tmp = operation_df.groupby('UID')['day'].agg([max,min,np.mean,gap]).reset_index()
    tmp.columns=['UID','op_day_max','op_day_min','op_day_mean', 'op_day_gap']
    op_fea = pd.merge(op_fea,tmp,on='UID',how='left')
    
    #Geo info
    geo = operation_df['geo_code'].apply(lambda x: ((x, x) if pd.isna(x) else geohash.decode(x)))
    operation_df['latitude'] = geo.apply(lambda x: x[0])
    operation_df['longtitude'] = geo.apply(lambda x: x[1])
    tmp = operation_df.groupby('UID')['latitude'].agg([max,min,np.mean,gap]).reset_index()
    tmp.columns = ['UID', 'op_latitude_max', 'op_latitude_min', 'op_latitude_mean', 'op_latitude_gap']
    op_fea = pd.merge(op_fea, tmp, on='UID', how='left')
    tmp = operation_df.groupby('UID')['longtitude'].agg([max,min,np.mean,gap]).reset_index()
    tmp.columns = ['UID', 'op_longtitude_max', 'op_longtitude_min', 'op_longtitude_mean', 'op_longtitude_gap']
    op_fea = pd.merge(op_fea, tmp, on='UID', how='left')
    #op_mode count
    op_fea = merge_count(operation_df,op_fea,'UID','mode','op_cnt')
    op_fea = merge_nunique(operation_df,op_fea,'UID','mode','op_mode_nunique')
    
    #op_time
    operation_df['op_hour'] = operation_df['time'].apply(lambda x:int(x.split(':')[0]))
    tmp = operation_df.groupby('UID')['op_hour'].agg([max,min,np.mean]).reset_index()
    tmp.columns=['UID','op_hour_max','op_hour_min','op_hour_mean']
    op_fea = pd.merge(op_fea,tmp,on='UID',how='left')
        
    #op_os
    for col in ['os','version','device1','device2','device_code1','device_code2','mac1','ip1','ip2','device_code3','mac2','wifi','ip1_sub','ip2_sub','geo_code']:
        op_fea = merge_nunique(operation_df,op_fea,'UID',col,'op_'+col+'_nunique')
    return op_fea

def get_trans_stats_fea(transaction_df):
    trans_counts = transaction_df.groupby('UID').size()
    
    trans_fea = transaction_df[['UID']].drop_duplicates()
        
    tmp = transaction_df.groupby('UID')['day'].agg([max,min,np.mean,gap]).reset_index()
    tmp.columns=['UID','trans_day_max','trans_day_min','trans_day_mean', 'trans_day_gap']
    trans_fea = pd.merge(trans_fea,tmp,on='UID',how='left')
    
    tmp = transaction_df.groupby('UID')['trans_amt'].agg([max,min,np.mean,gap]).reset_index()
    tmp.columns=['UID','trans_amt_max','trans_amt_min','trans_amt_mean', 'trans_amt_gap']
    trans_fea = pd.merge(trans_fea,tmp,on='UID',how='left')
    
    geo = transaction_df['geo_code'].apply(lambda x: ((x, x) if pd.isna(x) else geohash.decode(x)))
    transaction_df['latitude'] = geo.apply(lambda x: x[0])
    transaction_df['longtitude'] = geo.apply(lambda x: x[1])
    tmp = transaction_df.groupby('UID')['latitude'].agg([max,min,np.mean,gap]).reset_index()
    tmp.columns = ['UID', 'trans_latitude_max', 'trans_latitude_min', 'trans_latitude_mean', 'trans_latitude_gap']
    trans_fea = pd.merge(trans_fea, tmp, on='UID', how='left')
    tmp = transaction_df.groupby('UID')['longtitude'].agg([max,min,np.mean,gap]).reset_index()
    tmp.columns = ['UID', 'trans_longtitude_max', 'trans_longtitude_min', 'trans_longtitude_mean', 'trans_longtitude_gap']
    trans_fea = pd.merge(trans_fea, tmp, on='UID', how='left')
    
    #trans time
    transaction_df['trans_hour'] = transaction_df['time'].apply(lambda x:int(x.split(':')[0]))
    tmp = transaction_df.groupby('UID')['trans_hour'].agg([max,min,np.mean]).reset_index()
    tmp.columns=['UID','trans_hour_max','trans_hour_min','trans_hour_mean']
    trans_fea = pd.merge(trans_fea,tmp,on='UID',how='left')
    
    for col in ['trans_type1','merchant','code1','code2','acc_id1','device_code1','device_code2','device_code3','device1','device2','mac1','ip1','acc_id2','acc_id3','market_code','ip1_sub','geo_code']:
        trans_fea = merge_nunique(transaction_df,trans_fea,'UID',col,'trans_'+col+'_nunique')
    for col in ['trans_type2','market_type']:
        trans_fea = merge_nunique(transaction_df,trans_fea,'UID',col,'trans_'+col+'_nunique')
        
    return trans_fea


In [221]:
def get_trans_counted_features(transaction_df):
    trans_counts = transaction_df.groupby('UID').size()
    trans_fea = transaction_df[['UID']].drop_duplicates()
#     trans_fea = merge_value_count(transaction_df,trans_fea,['UID','day'], 'trans_day', divisor=trans_counts)
    #trans_channel
    trans_fea = merge_value_count(transaction_df,trans_fea,['UID','channel'], 'trans_channel', divisor=trans_counts)
    trans_fea = merge_count(transaction_df,trans_fea,'UID','channel','trans_cnt')
    trans_fea = merge_nunique(transaction_df,trans_fea,'UID','channel','trans_channel_nunique')
    transaction_df['trans_hour'] = transaction_df['time'].apply(lambda x:int(x.split(':')[0]))
    trans_fea = merge_value_count(transaction_df,trans_fea,['UID','trans_hour'], divisor=trans_counts)
    for col in ['trans_type2','market_type']:
        trans_fea = merge_value_count(transaction_df,trans_fea,['UID',col], 'trans_%s'%col, divisor=trans_counts)
        
    trans_fea = merge_value_count(transaction_df,trans_fea,['UID','day_in_week'], 'trans_week_day', divisor=trans_counts)

    trans_fea['trans_on_weekend'] = trans_fea[['trans_week_day_%d'%i for i in range(6, 8)]].sum(axis=1)
    trans_fea['trans_on_weekday'] = trans_fea[['trans_week_day_%d'%i for i in range(1, 6)]].sum(axis=1)
    
    
    ratio_list = [
        ('merchant', 'merchant'),
        ('amt_src1', 'amt_src1'),
        ('device2', 'trans_device2'),
        ('acc_id2', 'acc_id2'),
        ('acc_id3', 'acc_id3')
    ]
    
    for df_name, out_name in ratio_list:
        ratios = get_black_ratio(transaction_df, df_name, out_name)
        trans_fea = trans_fea.merge(ratios, on='UID', how='left')
    return trans_fea
    
    
def get_op_counted_features(operation_df):
    op_counts = operation_df.groupby('UID').size()
    
    op_fea = operation_df[['UID']].drop_duplicates()
#     op_fea = merge_value_count(operation_df,op_fea,['UID','day'],'op_days', divisor=op_counts)
    #success count
    op_fea = merge_count(operation_df[operation_df.success==0],op_fea,'UID','mode','op_fail_cnt')
    op_fea = merge_count(operation_df[operation_df.success==1],op_fea,'UID','mode','op_success_cnt')
    op_fea['op_fail_cnt'] = op_fea['op_fail_cnt'].fillna(0)
    op_fea['op_success_cnt'] = op_fea['op_success_cnt'].fillna(0)
    op_fea = merge_value_count(operation_df,op_fea,['UID','op_hour'], divisor=op_counts) 
    op_fea = merge_value_count(operation_df,op_fea,['UID','day_in_week'], 'op_week_day', divisor=op_counts) 

    op_fea['op_on_weekend'] = op_fea[['op_week_day_%d'%i for i in range(6, 8)]].sum(axis=1)
    op_fea['op_on_weekday'] = op_fea[['op_week_day_%d'%i for i in range(1, 6)]].sum(axis=1)
    
    pop_count = [('wifi', 'op_wifi_pop'), ('os', 'op_os_pop'), ('version', 'op_version_pop')]
    for df_name, out_name in pop_count:
        pop = getPopularityCount(operation_df, df_name, out_name)
        op_fea = pd.merge(op_fea,pop,on='UID',how='left')
    
    ratio_list = [
        ('device2', 'op_device2'),
    ]
    
    for df_name, out_name in ratio_list:
        ratios = get_black_ratio(operation_df, df_name, out_name)
        op_fea = op_fea.merge(ratios, on='UID', how='left')
    
    return op_fea

In [218]:
#Stats
operation_features_stats = get_op_stats_fea(operation_df)
transaction_features_stats = get_trans_stats_fea(transaction_df)

In [219]:
operation_test_stats = get_op_stats_fea(operation_test)
transaction_test_stats = get_trans_stats_fea(transaction_test)

In [222]:
#counted data
operation_features_counted = get_op_counted_features(operation_df)
transaction_features_counted = get_trans_counted_features(transaction_df)

In [223]:
operation_test_counted = get_op_counted_features(operation_test)
transaction_test_counted = get_trans_counted_features(transaction_test)

In [226]:
operation_features_counted

,UID,op_fail_cnt,op_success_cnt,op_hour_0,op_hour_1,op_hour_2,op_hour_3,op_hour_4,op_hour_5,op_hour_6,...,op_wifi_pop_max,op_os_pop_max,op_os_pop_min,op_os_pop_max,op_version_pop_max,op_version_pop_min,op_version_pop_max,max_op_device2_black_ratio,min_op_device2_black_ratio,avg_op_device2_black_ratio
0,10035,0.0,112.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,...,1.000000,24054,24054,24054.000000,17432.0,16753.0,17165.250000,0.144519,0.125261,0.125777
1,16264,4.0,229.0,0.112033,0.000000,0.0,0.0,0.000000,0.000000,0.087137,...,1.000000,24054,4211,21487.887967,17432.0,16753.0,17067.351852,0.144519,0.048000,0.086447
2,13162,3.0,13.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,...,1.000000,24054,16700,23621.411765,17432.0,16753.0,16965.187500,0.144519,0.016129,0.059301
3,21392,0.0,49.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,...,1.000000,24054,24054,24054.000000,17432.0,16753.0,17154.857143,0.144519,0.038168,0.055531
4,18599,0.0,112.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,...,NaN,24054,16700,21301.385475,1020.0,268.0,758.142857,0.144519,0.144519,0.144519
5,20499,2.0,162.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,...,1.000000,24054,4211,22692.605714,17432.0,16753.0,17134.356164,0.144519,0.022222,0.044585
6,21634,10.0,29.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.025641,...,65.520000,24054,4211,23036.410256,17432.0,16753.0,17266.837838,0.962366,0.144519,0.703403
7,11305,2.0,35.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,...,1.000000,24054,16700,23336.536585,17432.0,16753.0,17175.081081,0.144519,0.000000,0.042298
8,15270,15.0,68.0,0.313953,0.000000,0.0,0.0,0.000000,0.000000,0.000000,...,1.000000,24054,179,18166.209302,17432.0,4004.0,16887.017241,0.147541,0.097436,0.141208
9,10655,0.0,32.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,...,NaN,24054,24054,24054.000000,17432.0,16753.0,16901.531250,0.144519,0.023810,0.053987


In [224]:
train_features = operation_features_stats.merge(transaction_features_stats, on='UID', how='outer')
train_features = train_features.merge(operation_features_counted, on='UID', how='outer')
train_features = train_features.merge(transaction_features_counted, on='UID', how='outer')
train_data = train_features.merge(label,on='UID',how='left')
train_data = train_data.drop('UID', axis=1)
test_features = operation_test_stats.merge(transaction_test_stats, on='UID', how='outer').sort_values(['UID'])
test_features = test_features.merge(operation_test_counted, on='UID', how='outer')
test_features = test_features.merge(transaction_test_counted, on='UID', how='outer').sort_values(['UID'])

UIDs = test_features['UID']
test_features = test_features.drop('UID', axis=1)

columns_to_drop = [col for col in train_data.columns if col not in test_features.columns and col != 'Tag']
train_data = train_data.drop(columns_to_drop, axis=1)
columns_to_drop = [col for col in test_features.columns if col not in train_data.columns and col != 'Tag']
test_features = test_features.drop(columns_to_drop, axis=1)

#Fill nan with average
train_data = train_data.fillna(train_data.mean())
test_features = test_features.fillna(train_data.drop('Tag', axis=1).mean())

/usr/local/lib/python3.7/site-packages/pandas/core/generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/usr/local/lib/python3.7/site-packages/pandas/core/frame.py:4034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


KeyboardInterrupt: 

In [37]:
import sklearn.preprocessing
def preprocess(x):
    scaler = sklearn.preprocessing.MinMaxScaler()
    return scaler.fit_transform(x)

In [38]:
def tpr_scorer(estimator, x, y_true):
    y_predict = estimator.predict_proba(x)[:, 1]
    d = pd.DataFrame()
    d['prob'] = list(y_predict)
    d['y'] = list(y_true)
    d = d.sort_values(['prob'], ascending=[0])
    y = d.y
    
    PosAll = pd.Series(y).value_counts()[1]
    NegAll = pd.Series(y).value_counts()[0]
    pCumsum = d['y'].cumsum()
    nCumsum = np.arange(len(y)) - pCumsum + 1
    pCumsumPer = pCumsum / PosAll
    nCumsumPer = nCumsum / NegAll
    TR1 = pCumsumPer[abs(nCumsumPer-0.001).idxmin()]
    TR2 = pCumsumPer[abs(nCumsumPer-0.005).idxmin()]
    TR3 = pCumsumPer[abs(nCumsumPer-0.01).idxmin()]
    return 0.4 * TR1 + 0.3 * TR2 + 0.3 * TR3

In [79]:
import sklearn.model_selection
import sklearn.metrics
import lightgbm as lgb
import sklearn.ensemble
import xgboost as xgb

def cv(x, y, params={}, splits=5):
    clf = lgb.LGBMClassifier(**params)
#     clf = sklearn.ensemble.RandomForestClassifier(n_estimators=100, n_jobs=-1)
#     clf = xgb.XGBClassifier(**params)
    kfold = sklearn.model_selection.StratifiedKFold(splits, shuffle=True)
    cv_score = sklearn.model_selection.cross_validate(clf, x, y, cv=kfold, scoring={
        'tpr': tpr_scorer,
        'accuracy': 'accuracy',
        'f1': 'f1_micro',
        'roc_auc': 'roc_auc'
    }, return_train_score=True)
    return cv_score

In [148]:
params = {
        'boosting_type': 'gbdt',
        'objective': 'binary',
        "num_leaves": 200,
        "max_depth": -1,
        "learning_rate": 0.1,
        'min_child_samples': 100,
        'n_estimators': 2000,
        'learning_rate': 0.05,
        'scale_pos_weight': float(label[label.Tag==0].shape[0]) / label[label.Tag==1].shape[0],
        'boost_from_average': True,
        'min_child_weight': 1e-3,
        "subsample_for_bin": 20000,
        'max_bin': 512,
        "metric": 'auc',
        'reg_alpha': 3,
        'reg_lambda': 5,
        'subsample': 0.9,
        'colsample_bytree':0.7, 
        'subsample_freq': 1,
        'n_jobs': -1,
}
# params = {
#         'booster':'gbtree',
#         'objective':'binary:logistic',
#         'stratified':True,
#         'max_depth':8,
#         'min_child_weight':1,
#         'gamma':3,
#         'subsample':0.8,#0.7
#         'colsample_bytree':0.6, 
#         'reg_lambda':3, 
#         'eta':0.05,
#         'seed':20,
#         'silent':1,
#         'eval_metric':'auc',
#         'n_jobs': 12,
#         'n_estimators': 400,
#         'early_stopping_round': 200,
#         'scale_pos_weight': float(label[label.Tag==0].shape[0]) / label[label.Tag==1].shape[0],
#         'verbosity': 2
# }

def run_cross_validation(x, y):
    cv_result = cv(x, y, params=params, splits=5)
    for scorer, score in cv_result.items():
        print('%s: %s' % (scorer, score))
        print('Average %s: %f' % (scorer, score.mean()))

In [149]:
run_cross_validation(train_data.drop('Tag', axis=1).fillna(train_data.drop('Tag', axis=1).mean()).values, train_data.loc[:, 'Tag'].values)

fit_time: [8.738029   7.99695277 8.78002143 9.0420742  8.86215925]
Average fit_time: 8.683847
score_time: [0.39169097 0.39002395 0.41341972 0.41088891 0.48071599]
Average score_time: 0.417348
test_tpr: [0.85414236 0.82508751 0.85939323 0.84714119 0.85029172]
Average test_tpr: 0.847211
train_tpr: [1. 1. 1. 1. 1.]
Average train_tpr: 1.000000
test_accuracy: [0.97514432 0.97065427 0.97771007 0.97241822 0.97481957]
Average test_accuracy: 0.974149
train_accuracy: [0.99967927 0.99963918 0.99979954 0.99959909 0.99975946]
Average train_accuracy: 0.999695
test_f1: [0.97514432 0.97065427 0.97771007 0.97241822 0.97481957]
Average test_f1: 0.974149
train_f1: [0.99967927 0.99963918 0.99979954 0.99959909 0.99975946]
Average train_f1: 0.999695
test_roc_auc: [0.99061869 0.98993666 0.99337087 0.98938154 0.99230019]
Average test_roc_auc: 0.991122
train_roc_auc: [0.99999997 0.99999997 0.99999996 0.99999999 0.99999995]
Average train_roc_auc: 1.000000


In [150]:
def train(x, y, params={}):
    clf = lgb.LGBMClassifier(**params)
#     clf = xgb.XGBClassifier(**params)
    clf.fit(x, y)
    return clf

In [151]:
model = train(train_data.drop('Tag', axis=1).values, train_data.loc[:, 'Tag'].values, params=params)

In [152]:
result = model.predict_proba(test_features.values)[:, 1]

In [153]:
(result > 0.5).sum() / len(result)

0.0967049169818578

In [154]:
tpr_scorer(model, train_data.drop('Tag', axis=1).fillna(train_data.drop('Tag', axis=1).mean()).values, train_data.loc[:, 'Tag'].values)

1.0

In [155]:
result_frame = pd.DataFrame()
result_frame['UID'] = UIDs
result_frame['Tag'] = result

In [156]:

result_frame.to_csv(res_path + 'res.csv', index=False)

In [157]:
import matplotlib.pyplot as pyplot

In [158]:
[(train_data.drop('Tag', axis=1).columns[i], model.feature_importances_[i]) for i in range(len(model.feature_importances_))]

[('op_day_max', 643),
 ('op_day_min', 570),
 ('op_day_mean', 2332),
 ('op_day_gap', 509),
 ('op_latitude_max', 625),
 ('op_latitude_min', 620),
 ('op_latitude_mean', 443),
 ('op_latitude_gap', 167),
 ('op_longtitude_max', 577),
 ('op_longtitude_min', 619),
 ('op_longtitude_mean', 589),
 ('op_longtitude_gap', 78),
 ('op_cnt', 1671),
 ('op_mode_nunique', 977),
 ('op_hour_max', 623),
 ('op_hour_min', 694),
 ('op_hour_mean', 1858),
 ('op_os_nunique', 135),
 ('op_version_nunique', 38),
 ('op_device1_nunique', 55),
 ('op_device2_nunique', 31),
 ('op_device_code1_nunique', 65),
 ('op_device_code2_nunique', 146),
 ('op_mac1_nunique', 93),
 ('op_ip1_nunique', 389),
 ('op_ip2_nunique', 67),
 ('op_device_code3_nunique', 86),
 ('op_mac2_nunique', 169),
 ('op_wifi_nunique', 319),
 ('op_ip1_sub_nunique', 476),
 ('op_ip2_sub_nunique', 48),
 ('op_geo_code_nunique', 92),
 ('trans_day_max', 448),
 ('trans_day_min', 495),
 ('trans_day_mean', 1474),
 ('trans_day_gap', 169),
 ('trans_amt_max', 658),
 ('tra